In [78]:
from google.colab import files
import pandas as pd

uploaded = files.upload()
df_server_1_equal = pd.read_csv('server_one_dataframe_division_mostly_low_discount.csv')

uploaded = files.upload()
df_server_2_equal = pd.read_csv('server_two_dataframe_division_mostly_high_discount.csv')

Saving server_one_dataframe_division_mostly_low_discount.csv to server_one_dataframe_division_mostly_low_discount.csv


Saving server_two_dataframe_division_mostly_high_discount.csv to server_two_dataframe_division_mostly_high_discount.csv


In [80]:
# CTGAN: https://github.com/sdv-dev/CTGAN
# !pip install ctgan

In [81]:
df_server_1_equal.head()

,Segment,Country,Product,Discount Band,Units Sold,Manufacturing Price,Sale Price,Gross Sales,Discounts,Sales,COGS,Profit,Date,Month Number,Month Name,Year
0,Government,United States of America,Amarilla,None,1143.0,260.0,7.0,8001.0,0.00,8001.00,5715.0,2286.00,01/10/2014,10,October,2014
1,Enterprise,United States of America,Carretera,Low,330.0,3.0,125.0,41250.0,412.50,40837.50,39600.0,1237.50,01/09/2013,9,September,2013
2,Enterprise,France,Paseo,Low,2988.0,10.0,125.0,373500.0,14940.00,358560.00,358560.0,0,01/07/2014,7,July,2014
3,Government,United States of America,VTT,Low,266.0,250.0,350.0,93100.0,1862.00,91238.00,69160.0,22078.00,01/12/2013,12,December,2013
4,Enterprise,Mexico,Paseo,Low,1823.0,10.0,125.0,227875.0,2278.75,225596.25,218760.0,6836.25,01/07/2014,7,July,2014


In [82]:
# Gross sales = units solds * sales price
# Gross sales to sales: sales = gross sales - discount
# COGS (cost of goods sold) = units sold * manufacturing price
# profit = sales - COGS
# month name from month number
# private feature is the discount band


# So generate units sold, sales price, discount, manufacturing price, month number, and year

columns_to_drop = ['Segment', 'Country', 'Product', 'Discount Band', 'Gross Sales', 'Sales', 'COGS', 'Profit', 'Date', 'Month Name']
df_server_1_filtered = df_server_1_equal.drop(columns = columns_to_drop)

columns_to_drop = ['Segment', 'Country', 'Product', 'Discount Band', 'Gross Sales', 'Sales', 'COGS', 'Profit', 'Date', 'Month Name']
df_server_2_filtered = df_server_2_equal.drop(columns = columns_to_drop)

In [83]:
print(len(df_server_1_filtered))

254


In [84]:
discrete_columns = ['Month Number', 'Year']

from ctgan import CTGAN

ctgan = CTGAN(epochs = 10)
ctgan.fit(df_server_1_filtered, discrete_columns)

/usr/local/lib/python3.10/dist-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'mi

In [85]:
df_server_1_synthetic = ctgan.sample(len(df_server_1_filtered))

In [86]:
# df_server_1_synthetic.head()
# Now units sold should be an integer, manufacturing, sales price, and discuounts should be > 0
import numpy as np

df_server_1_synthetic['Units Sold'] = np.ceil(df_server_1_synthetic['Units Sold'])
df_server_1_synthetic['Manufacturing Price'] = np.abs(df_server_1_synthetic['Manufacturing Price'])
df_server_1_synthetic['Sale Price'] = np.abs(df_server_1_synthetic['Sale Price'])
df_server_1_synthetic['Discounts'] = np.abs(df_server_1_synthetic['Discounts'])

In [87]:
import calendar

df_server_1_synthetic['Gross Sales'] = df_server_1_synthetic['Units Sold'] * df_server_1_synthetic['Sale Price']
df_server_1_synthetic['Sales'] = df_server_1_synthetic['Gross Sales'] - df_server_1_synthetic['Discounts']
df_server_1_synthetic['COGS'] = df_server_1_synthetic['Units Sold'] * df_server_1_synthetic['Manufacturing Price']
df_server_1_synthetic['Profit'] = df_server_1_synthetic['Sales'] - df_server_1_synthetic['COGS']
df_server_1_synthetic['Month Name'] = df_server_1_synthetic['Month Number'].apply(lambda x: calendar.month_name[x])

In [88]:
df_server_1_synthetic['Segment'] = df_server_1_equal['Segment']
df_server_1_synthetic['Country'] = df_server_1_equal['Country']
df_server_1_synthetic['Product'] = df_server_1_equal['Product']
df_server_1_synthetic['Discount Band'] = df_server_1_equal['Discount Band']
df_server_1_synthetic['Date'] = pd.to_datetime(df_server_1_synthetic['Year'].astype(str) + '-' + df_server_1_synthetic['Month Number'].astype(str) + '-1')
df_server_1_synthetic['Date'] = df_server_1_synthetic['Date'].dt.strftime('%d/%m/%Y')

In [89]:
df_server_1_synthetic = df_server_1_synthetic[df_server_1_equal.columns]

In [90]:
df_server_1_synthetic.head()

,Segment,Country,Product,Discount Band,Units Sold,Manufacturing Price,Sale Price,Gross Sales,Discounts,Sales,COGS,Profit,Date,Month Number,Month Name,Year
0,Government,United States of America,Amarilla,None,1536.0,283.341740,368.306311,5.657185e+05,2019.480532,5.636990e+05,435212.913391,1.284861e+05,01/12/2014,12,December,2014
1,Enterprise,United States of America,Carretera,Low,170.0,281.309576,83.184927,1.414144e+04,58247.553241,-4.410612e+04,47822.627868,-9.192874e+04,01/04/2014,4,April,2014
2,Enterprise,France,Paseo,Low,573.0,111.965107,34.538240,1.979041e+04,24713.734715,-4.923323e+03,64156.006328,-6.907933e+04,01/12/2013,12,December,2013
3,Government,United States of America,VTT,Low,2873.0,21.353995,161.406778,4.637217e+05,20584.973617,4.431367e+05,61350.028319,3.817867e+05,01/04/2014,4,April,2014
4,Enterprise,Mexico,Paseo,Low,4584.0,8.969675,384.240450,1.761358e+06,27335.976876,1.734022e+06,41116.991303,1.692905e+06,01/06/2014,6,June,2014


In [92]:
# df_server_1_synthetic.to_csv('server_one_synthetic_dataset_mostly_low_discount.csv', index = False)
# files.download('server_one_synthetic_dataset_mostly_low_discount.csv')

In [93]:
ctgan = CTGAN(epochs = 10)
ctgan.fit(df_server_2_filtered, discrete_columns)

/usr/local/lib/python3.10/dist-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'mi

In [94]:
df_server_2_synthetic = ctgan.sample(len(df_server_2_filtered))

In [95]:
df_server_2_synthetic['Units Sold'] = np.ceil(df_server_2_synthetic['Units Sold'])
df_server_2_synthetic['Manufacturing Price'] = np.abs(df_server_2_synthetic['Manufacturing Price'])
df_server_2_synthetic['Sale Price'] = np.abs(df_server_2_synthetic['Sale Price'])
df_server_2_synthetic['Discounts'] = np.abs(df_server_2_synthetic['Discounts'])

In [96]:
df_server_2_synthetic['Gross Sales'] = df_server_2_synthetic['Units Sold'] * df_server_2_synthetic['Sale Price']
df_server_2_synthetic['Sales'] = df_server_2_synthetic['Gross Sales'] - df_server_2_synthetic['Discounts']
df_server_2_synthetic['COGS'] = df_server_2_synthetic['Units Sold'] * df_server_2_synthetic['Manufacturing Price']
df_server_2_synthetic['Profit'] = df_server_2_synthetic['Sales'] - df_server_2_synthetic['COGS']
df_server_2_synthetic['Month Name'] = df_server_2_synthetic['Month Number'].apply(lambda x: calendar.month_name[x])

In [97]:
df_server_2_synthetic['Segment'] = df_server_2_equal['Segment']
df_server_2_synthetic['Country'] = df_server_2_equal['Country']
df_server_2_synthetic['Product'] = df_server_2_equal['Product']
df_server_2_synthetic['Discount Band'] = df_server_2_equal['Discount Band']
df_server_2_synthetic['Date'] = pd.to_datetime(df_server_2_synthetic['Year'].astype(str) + '-' + df_server_2_synthetic['Month Number'].astype(str) + '-1')
df_server_2_synthetic['Date'] = df_server_2_synthetic['Date'].dt.strftime('%d/%m/%Y')

In [98]:
df_server_2_synthetic = df_server_2_synthetic[df_server_2_equal.columns]

In [99]:
df_server_2_synthetic.head()

,Segment,Country,Product,Discount Band,Units Sold,Manufacturing Price,Sale Price,Gross Sales,Discounts,Sales,COGS,Profit,Date,Month Number,Month Name,Year
0,Government,United States of America,Amarilla,None,1730.0,16.810230,160.696319,278004.631129,5498.424159,272506.206970,29081.697832,243424.509138,01/01/2014,1,January,2014
1,Enterprise,United States of America,Carretera,Low,1007.0,228.654744,0.456610,459.806007,82539.846511,-82080.040505,230255.327065,-312335.367570,01/07/2013,7,July,2013
2,Enterprise,France,Paseo,Low,923.0,90.223797,7.324572,6760.580101,22872.166852,-16111.586750,83276.564379,-99388.151129,01/02/2013,2,February,2013
3,Government,United States of America,VTT,Low,1321.0,101.450069,37.296781,49269.047638,6093.133597,43175.914041,134015.541746,-90839.627705,01/02/2014,2,February,2014
4,Enterprise,Mexico,Paseo,Low,465.0,112.146608,301.631198,140258.507207,28578.023230,111680.483977,52148.172844,59532.311133,01/04/2013,4,April,2013


In [100]:
df_server_2_synthetic.to_csv('server_two_synthetic_dataset_mostly_high_discount.csv', index = False)
files.download('server_two_synthetic_dataset_mostly_high_discount.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [38]:
uploaded = files.upload()
df_server_1_north_america = pd.read_csv('server_one_dataframe_north_america.csv')

uploaded = files.upload()
df_server_2_europe = pd.read_csv('server_two_dataframe_europe.csv')

Saving server_one_dataframe_north_america.csv to server_one_dataframe_north_america.csv


Saving server_two_dataframe_europe.csv to server_two_dataframe_europe.csv


In [39]:
columns_to_drop = ['Segment', 'Country', 'Product', 'Discount Band', 'Gross Sales', 'Sales', 'COGS', 'Profit', 'Date', 'Month Name']
df_server_1_north_america_filtered = df_server_1_north_america.drop(columns = columns_to_drop)

columns_to_drop = ['Segment', 'Country', 'Product', 'Discount Band', 'Gross Sales', 'Sales', 'COGS', 'Profit', 'Date', 'Month Name']
df_server_2_europe_filtered = df_server_2_europe.drop(columns = columns_to_drop)

In [40]:
discrete_columns = ['Month Number', 'Year']

from ctgan import CTGAN

ctgan = CTGAN(epochs = 10)
ctgan.fit(df_server_1_north_america_filtered, discrete_columns)

/usr/local/lib/python3.10/dist-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'mi

In [41]:
df_server_1_north_america_synthetic = ctgan.sample(len(df_server_1_north_america_filtered))

In [42]:
df_server_1_north_america_synthetic['Units Sold'] = np.ceil(df_server_1_north_america_synthetic['Units Sold'])
df_server_1_north_america_synthetic['Manufacturing Price'] = np.abs(df_server_1_north_america_synthetic['Manufacturing Price'])
df_server_1_north_america_synthetic['Sale Price'] = np.abs(df_server_1_north_america_synthetic['Sale Price'])
df_server_1_north_america_synthetic['Discounts'] = np.abs(df_server_1_north_america_synthetic['Discounts'])

In [43]:
df_server_1_north_america_synthetic['Gross Sales'] = df_server_1_north_america_synthetic['Units Sold'] * df_server_1_north_america_synthetic['Sale Price']
df_server_1_north_america_synthetic['Sales'] = df_server_1_north_america_synthetic['Gross Sales'] - df_server_1_north_america_synthetic['Discounts']
df_server_1_north_america_synthetic['COGS'] = df_server_1_north_america_synthetic['Units Sold'] * df_server_1_north_america_synthetic['Manufacturing Price']
df_server_1_north_america_synthetic['Profit'] = df_server_1_north_america_synthetic['Sales'] - df_server_1_north_america_synthetic['COGS']
df_server_1_north_america_synthetic['Month Name'] = df_server_1_north_america_synthetic['Month Number'].apply(lambda x: calendar.month_name[x])

In [44]:
df_server_1_north_america_synthetic['Segment'] = df_server_1_north_america['Segment']
df_server_1_north_america_synthetic['Country'] = df_server_1_north_america['Country']
df_server_1_north_america_synthetic['Product'] = df_server_1_north_america['Product']
df_server_1_north_america_synthetic['Discount Band'] = df_server_1_north_america['Discount Band']
df_server_1_north_america_synthetic['Date'] = pd.to_datetime(df_server_1_north_america_synthetic['Year'].astype(str) + '-' + df_server_1_north_america_synthetic['Month Number'].astype(str) + '-1')
df_server_1_north_america_synthetic['Date'] = df_server_1_north_america_synthetic['Date'].dt.strftime('%d/%m/%Y')

In [45]:
df_server_1_north_america_synthetic = df_server_1_north_america_synthetic[df_server_1_north_america.columns]

In [46]:
df_server_1_north_america_synthetic.head()

,Segment,Country,Product,Discount Band,Units Sold,Manufacturing Price,Sale Price,Gross Sales,Discounts,Sales,COGS,Profit,Date,Month Number,Month Name,Year
0,Government,Canada,Carretera,None,1970.0,258.726455,359.018315,707266.080698,35940.902447,671325.178251,509691.116074,161634.062176,01/01/2014,1,January,2014
1,Midmarket,Mexico,Carretera,None,1002.0,291.937871,136.284332,136556.900851,5056.141995,131500.758855,292521.746507,-161020.987652,01/09/2013,9,September,2013
2,Channel Partners,Canada,Montana,None,2039.0,35.463074,97.555445,198915.551595,136164.915991,62750.635604,72309.207023,-9558.571419,01/06/2014,6,June,2014
3,Midmarket,Mexico,Montana,None,1052.0,149.067149,29.340548,30866.256908,10328.434600,20537.822309,156818.641102,-136280.818794,01/07/2013,7,July,2013
4,Enterprise,Canada,Montana,None,1181.0,32.284835,26.267986,31022.490976,7870.304315,23152.186661,38128.390315,-14976.203655,01/01/2013,1,January,2013


In [47]:
# df_server_1_north_america_synthetic.to_csv('server_one_synthetic_dataset_north_america.csv', index = False)
# files.download('server_one_synthetic_dataset_north_america.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [69]:
ctgan = CTGAN(epochs = 10)
ctgan.fit(df_server_2_europe_filtered, discrete_columns)

/usr/local/lib/python3.10/dist-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'mi

In [70]:
df_server_2_europe_synthetic = ctgan.sample(len(df_server_2_europe_filtered))

In [71]:
df_server_2_europe_synthetic['Units Sold'] = np.ceil(df_server_2_europe_synthetic['Units Sold'])
df_server_2_europe_synthetic['Manufacturing Price'] = np.abs(df_server_2_europe_synthetic['Manufacturing Price'])
df_server_2_europe_synthetic['Sale Price'] = np.abs(df_server_2_europe_synthetic['Sale Price'])
df_server_2_europe_synthetic['Discounts'] = np.abs(df_server_2_europe_synthetic['Discounts'])

In [72]:
df_server_2_europe_synthetic['Gross Sales'] = df_server_2_europe_synthetic['Units Sold'] * df_server_2_europe_synthetic['Sale Price']
df_server_2_europe_synthetic['Sales'] = df_server_2_europe_synthetic['Gross Sales'] - df_server_2_europe_synthetic['Discounts']
df_server_2_europe_synthetic['COGS'] = df_server_2_europe_synthetic['Units Sold'] * df_server_2_europe_synthetic['Manufacturing Price']
df_server_2_europe_synthetic['Profit'] = df_server_2_europe_synthetic['Sales'] - df_server_2_europe_synthetic['COGS']
df_server_2_europe_synthetic['Month Name'] = df_server_2_europe_synthetic['Month Number'].apply(lambda x: calendar.month_name[x])

In [73]:
df_server_2_europe_synthetic['Segment'] = df_server_2_europe['Segment']
df_server_2_europe_synthetic['Country'] = df_server_2_europe['Country']
df_server_2_europe_synthetic['Product'] = df_server_2_europe['Product']
df_server_2_europe_synthetic['Discount Band'] = df_server_2_europe['Discount Band']
df_server_2_europe_synthetic['Date'] = pd.to_datetime(df_server_2_europe_synthetic['Year'].astype(str) + '-' + df_server_2_europe_synthetic['Month Number'].astype(str) + '-1')
df_server_2_europe_synthetic['Date'] = df_server_2_europe_synthetic['Date'].dt.strftime('%d/%m/%Y')

In [74]:
df_server_2_europe_synthetic = df_server_2_europe_synthetic[df_server_2_europe.columns]

In [75]:
df_server_2_europe_synthetic.head()

,Segment,Country,Product,Discount Band,Units Sold,Manufacturing Price,Sale Price,Gross Sales,Discounts,Sales,COGS,Profit,Date,Month Number,Month Name,Year
0,Government,Germany,Carretera,None,2187.0,205.575004,109.131289,238670.129769,51616.455186,187053.674583,449592.533984,-262538.859401,01/11/2013,11,November,2013
1,Midmarket,France,Carretera,None,2789.0,13.850231,102.422008,285654.980462,4775.381853,280879.598610,38628.294815,242251.303794,01/05/2013,5,May,2013
2,Midmarket,Germany,Carretera,None,1539.0,258.691977,9.155531,14090.362240,8431.185341,5659.176899,398126.953060,-392467.776161,01/07/2013,7,July,2013
3,Government,Germany,Carretera,None,1485.0,202.466321,7.401625,10991.412399,4108.344187,6883.068212,300662.486582,-293779.418370,01/11/2013,11,November,2013
4,Midmarket,Germany,Montana,None,3488.0,17.245425,253.267329,883396.442161,29616.089988,853780.352173,60152.042171,793628.310002,01/02/2013,2,February,2013


In [77]:
# df_server_2_europe_synthetic.to_csv('server_two_synthetic_dataset_europe.csv', index = False)
# files.download('server_two_synthetic_dataset_europe.csv')